In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Input
from keras.models import Model
from keras.optimizers import SGD, Adam

import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!nvcc -V

In [ ]:
X_train = pd.read_csv('../input/X_train_sat6.csv', 
                      header=None, 
                      nrows=20000).values.astype('int16').reshape(-1, 28, 28, 4)[:, :, :, :3]
y_train = pd.read_csv('../input/y_train_sat6.csv', header=None, nrows=20000).values.astype('int16')

X_test = pd.read_csv('../input/X_test_sat6.csv', 
                     header=None, 
                    nrows=5000).values.astype('int16').reshape(-1, 28, 28, 4)[:, :, :, :3]
y_test = pd.read_csv('../input/y_test_sat6.csv', header=None, nrows=5000).values.astype('int16')


In [ ]:
#plt.imshow(cv2.resize(X_train[0].astype('float32'), (224, 224), interpolation=cv2.INTER_CUBIC).astype('int16'))

In [ ]:
X_train[0].shape, X_train[0].dtype

In [ ]:
new_shape = 112

In [ ]:
X_train_prep = np.array([cv2.resize(i.astype('float32'), (new_shape, new_shape), interpolation=cv2.INTER_CUBIC).astype('int16') for i in X_train])

In [ ]:
#X_train_vgg = X_train_prep/255.
#X_train_vgg = X_train_vgg.astype('float32')
X_train_vgg = preprocess_input(X_train_prep).astype('float32')

In [ ]:
X_train_vgg.shape

In [ ]:
vgg = VGG16(include_top=False, input_shape=(new_shape, new_shape, 3))
new_gap = GlobalAveragePooling2D(name='new_flatten')(vgg.output)
dense = Dense(256, name='new_fc1')(new_gap)
dense = Dense(128, name='new_fc2')(dense)
dense6 = Dense(6, name='output')(dense)

model = Model(inputs=[vgg.input], outputs=[dense6])

for layer in vgg.layers:
    layer.trainable = False
    
base_lr = 0.01
momentum = 0.9
decay_weight = 0.0005

optimizer = Adam(lr=base_lr, 
                decay=decay_weight
                )

model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy', 
              metrics=['acc'])

#model.summary()

In [ ]:
model.fit(X_train_vgg, y_train, 
          epochs=500, 
          validation_split=0.1,
          batch_size = 32, 
          shuffle=True, 
          verbose=1)